In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dataset-Capex.csv to Dataset-Capex.csv


In [ ]:
CapexDS = pd.read_csv("Dataset-Capex.csv")

In [ ]:
CapexDS.head()

,Project_ID,Allocated_Budget,Actual_Expenditure,Variance,Project_Phase
0,PROJ_1,2937669.88,1556219.07,-13082.66,Planning
1,PROJ_2,3402664.40,3050159.23,-12413.58,Execution
2,PROJ_3,1958590.02,4389855.71,-38563.31,Planning
3,PROJ_4,4969562.89,2619133.77,-33535.48,Execution
4,PROJ_5,2976278.14,1387431.76,7734.75,Execution


In [ ]:
CapexDS[['Allocated_Budget', 'Actual_Expenditure']] = CapexDS[['Allocated_Budget', 'Actual_Expenditure']].apply(pd.to_numeric, errors='coerce')
CapexDS.fillna(0, inplace=True)

In [ ]:
CapexDS.head()

,Project_ID,Allocated_Budget,Actual_Expenditure,Variance,Project_Phase,Budget_Status,Adjusted_Budget,Cost_Overrun,Project_Phase_Encoded,Predicted_Cost_Overrun,Predicted_Final_Cost
0,PROJ_1,2937669.88,1556219.07,1381450.81,Planning,Under Budget,2937669.880,-1381450.81,2,-8.735284e+05,2.064141e+06
1,PROJ_2,3402664.40,3050159.23,352505.17,Execution,Under Budget,3402664.400,-352505.17,1,-8.735284e+05,2.529136e+06
2,PROJ_3,1958590.02,4389855.71,-2431265.69,Planning,Over Budget,2056519.521,2431265.69,2,4.158832e+05,2.374473e+06
3,PROJ_4,4969562.89,2619133.77,2350429.12,Execution,Under Budget,4969562.890,-2350429.12,1,-2.524084e+06,2.445479e+06
4,PROJ_5,2976278.14,1387431.76,1588846.38,Execution,Under Budget,2976278.140,-1588846.38,1,-8.735284e+05,2.102750e+06


In [ ]:
CapexDS['Variance'] = CapexDS['Allocated_Budget'] - CapexDS['Actual_Expenditure']
CapexDS['Budget_Status'] = np.where(CapexDS['Variance'] < 0, 'Over Budget', 'Under Budget')

In [ ]:
def adjust_budget(row, increase_factor=1.05, decrease_factor=0.95):
    """Increase budget by a factor for overruns & decrease for under-budget projects"""
    if row['Budget_Status'] == 'Over Budget':
        return row['Allocated_Budget'] * increase_factor
    elif row['Budget_Status'] == 'Under Budget':
        return row['Allocated_Budget'] * decrease_factor
    return row['Allocated_Budget']

CapexDS['Adjusted_Budget'] = CapexDS.apply(adjust_budget, axis=1)

In [ ]:
# Scenario Planning: Adjusting budgets to minimize overruns
def adjust_budget(row, adjustment_factor=1.05):
    """Increase budget by 5% for over-budget projects"""
    if row['Budget_Status'] == 'Over Budget':
        return row['Allocated_Budget'] * adjustment_factor
    return row['Allocated_Budget']

CapexDS['Adjusted_Budget'] = CapexDS.apply(adjust_budget, axis=1)

In [ ]:
summary = CapexDS.groupby('Project_Phase').agg({
    'Allocated_Budget': 'sum',
    'Actual_Expenditure': 'sum',
    'Variance': 'sum'
}).reset_index()

In [ ]:
print("\nCost Variance Analysis:\n", CapexDS[['Project_ID', 'Allocated_Budget', 'Actual_Expenditure', 'Variance', 'Budget_Status']])
print("\nScenario Planning (Adjusted Budgets):\n", CapexDS[['Project_ID', 'Allocated_Budget', 'Adjusted_Budget', 'Variance']])
print("\nSummary by Project Phase:\n", summary)


Cost Variance Analysis:
    Project_ID  Allocated_Budget  Actual_Expenditure    Variance Budget_Status
0      PROJ_1        2937669.88          1556219.07  1381450.81  Under Budget
1      PROJ_2        3402664.40          3050159.23   352505.17  Under Budget
2      PROJ_3        1958590.02          4389855.71 -2431265.69   Over Budget
3      PROJ_4        4969562.89          2619133.77  2350429.12  Under Budget
4      PROJ_5        2976278.14          1387431.76  1588846.38  Under Budget
..        ...               ...                 ...         ...           ...
95    PROJ_96        1701096.37          3615280.62 -1914184.25   Over Budget
96    PROJ_97        2734982.27          1453803.78  1281178.49  Under Budget
97    PROJ_98         757522.70          1924122.86 -1166600.16   Over Budget
98    PROJ_99         590917.91          2319526.38 -1728608.47   Over Budget
99   PROJ_100        2030513.29          2503108.31  -472595.02   Over Budget

[100 rows x 5 columns]

Scenario Plan

In [ ]:
pip install bayesian-optimization

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from scipy.stats import randint
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor
from bayes_opt import BayesianOptimization

In [ ]:
CapexDS[['Allocated_Budget', 'Actual_Expenditure']] = CapexDS[['Allocated_Budget', 'Actual_Expenditure']].apply(pd.to_numeric, errors='coerce')
CapexDS.fillna(0, inplace=True)

In [ ]:
CapexDS['Cost_Overrun'] = CapexDS['Actual_Expenditure'] - CapexDS['Allocated_Budget']

In [ ]:
if 'Project_Phase' in CapexDS.columns:
    encoder = LabelEncoder()
    CapexDS['Project_Phase_Encoded'] = encoder.fit_transform(CapexDS['Project_Phase'])

In [ ]:
features = ['Allocated_Budget']  # Start with budget, add more if available
if 'Project_Phase_Encoded' in CapexDS.columns:
    features.append('Project_Phase_Encoded')

X = CapexDS[features]
y = CapexDS['Cost_Overrun']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42,criterion="friedman_mse")
model.fit(X_train, y_train)

RandomForestRegressor(criterion='friedman_mse', random_state=42)

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\nModel Evaluation:\nR² Score: {r2:.2f}\nMAE: {mae:.2f}\nRMSE: {rmse:.2f}")



Model Evaluation:
R² Score: 0.54
MAE: 1684911.68
RMSE: 1912098.17


In [ ]:
# ---------- 6. Predict Future Overruns ----------
CapexDS['Predicted_Cost_Overrun'] = model.predict(X)
CapexDS['Predicted_Final_Cost'] = CapexDS['Allocated_Budget'] + CapexDS['Predicted_Cost_Overrun']

In [ ]:
CapexDS.to_csv("CapEx_Predictions.csv", index=False)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ---------- 3. Train-Test Split ----------
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

# ---------- 4. Hyperparameter Tuning using GridSearch ----------
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [1, 3, 5],
    'learning_rate': [1.54, 0.2, 0.08]

}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# ---------- 5. Evaluate Model ----------
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\nOptimized Model Evaluation:\nR² Score: {r2:.2f}\nMAE: {mae:.2f}\nRMSE: {rmse:.2f}")
print(f"Best Parameters: {grid_search.best_params_}")

# ---------- 6. Predict Future Overruns ----------
CapexDS['Predicted_Cost_Overrun'] = best_model.predict(X_scaled)
CapexDS['Predicted_Final_Cost'] = CapexDS['Allocated_Budget'] + CapexDS['Predicted_Cost_Overrun']



Optimized Model Evaluation:
R² Score: 0.51
MAE: 1763646.75
RMSE: 1977388.17
Best Parameters: {'learning_rate': 0.08, 'max_depth': 1, 'n_estimators': 100}


In [ ]:
# Define the parameter distribution to sample from
param_dist = {
    'max_depth': randint(1, 20),
    'min_samples_split': randint(1, 20),
    'min_samples_leaf': randint(1, 20)
}
dtree_reg = DecisionTreeRegressor(random_state=42)
random_search = RandomizedSearchCV(dtree_reg, param_distributions=param_dist,
                                   n_iter=100, cv=5, random_state=42)
random_search.fit(X_train, y_train)
best_params_random = random_search.best_params_
best_score_random = random_search.best_score_

print(f"Best Parameters (Random Search): {best_params_random}")
print(f"Best Score (Random Search): {best_score_random}")

Best Parameters (Random Search): {'max_depth': 11, 'min_samples_leaf': 11, 'min_samples_split': 4}
Best Score (Random Search): 0.37825128701770555


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
55 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

In [ ]:
# Define the function to optimize using cross-validation
def dtree_cv(max_depth, min_samples_split, min_samples_leaf):
    # Define the model with the parameters to be optimized
    estimator = DecisionTreeRegressor(
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        random_state=1
    )
    cval = cross_val_score(estimator, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    return cval.mean() # The optimizer tries to maximize the function, so we negate the score to minimize it

# Define the parameter bounds
param_bounds = {
    'max_depth': (1, 20),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (6, 20)
}

optimizer = BayesianOptimization(
    f=dtree_cv,
    pbounds=param_bounds,
    random_state=1,
)

optimizer.maximize(n_iter=10, init_points=5) # Bayesian optimization
best_params_bayes = optimizer.max['params']
best_params_bayes['max_depth'] = int(best_params_bayes['max_depth'])
best_params_bayes['min_samples_split'] = int(best_params_bayes['min_samples_split'])
best_params_bayes['min_samples_leaf'] = int(best_params_bayes['min_samples_leaf'])
best_score_bayes = optimizer.max['target']

print(f"Best Parameters (Bayesian Optimization): {best_params_bayes}")
print(f"Best Score (Bayesian Optimization): {best_score_bayes}")


|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | -2.407e+1 | 8.923     | 16.08     | 2.002     |
| 2         | -2.609e+1 | 6.744     | 8.055     | 3.662     |
| 3         | -2.469e+1 | 4.539     | 10.84     | 9.142     |
| 4         | -2.369e+1 | 11.24     | 11.87     | 14.33     |
| 5         | -2.867e+1 | 4.885     | 18.29     | 2.493     |
| 6         | -2.369e+1 | 11.2      | 11.47     | 14.53     |
| 7         | -2.44e+12 | 7.635     | 12.21     | 14.4      |
| 8         | -2.424e+1 | 12.91     | 14.98     | 2.234     |
| 9         | -2.424e+1 | 12.49     | 13.68     | 18.04     |
| 10        | -2.469e+1 | 15.54     | 10.74     | 13.67     |
| 11        | -2.469e+1 | 10.84     | 10.19     | 10.88     |
| 12        | -2.613e+1 | 9.917     | 8.405     | 18.99     |
| 13        | -2.407e+1 | 10.26     | 16.26     | 15.57     |
| 14        | -2.407e+1 | 14.43     | 16.6      | 14.44     |
| 15    